In [ ]:
#импорт библиотек
import matplotlib.pyplot as plt
import math
import numpy as np
import datetime
import time
import pandas as pd
import getpass
import random
import cv2
import os
import paho.mqtt.client as mqtt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from shapely.geometry import LineString, Polygon

In [ ]:
#импорт собственных модулей
from Lamp import *
from Cloude import*
from Flower import *
from Sun import *
from angles import*
%matplotlib notebook

In [ ]:
def update_field(sun, cloud_list): #функция обновления поля
    global latitude, longitude, utc_offset,  hour, minute
    SEA = math.radians(getSEA(latitude, longitude, utc_offset,  hour, minute)) #угол подъема солнца
    AZ = math.radians(getAZ(latitude, longitude, utc_offset, hour, minute)) #азимутальный угол
    minute+=1
    sun.update_position(SEA, AZ) #обновление позиции солнца
    [x.move() for x in cloude_list] #применение функции move ко всему списку облаков

In [ ]:
#Алгоритм поиска точки пересечения
def intersection(Segment, Plane):
    P0, P1 = np.array(Segment.coords)
    Q0, Q1, Q2 = np.array(Plane.exterior)[:-1]   

    #Вектора на плоскости
    vector_plane_1 = Q1 - Q0
    vector_plane_2 = Q2 - Q0

    #Вектор нормали
    n  = np.cross(vector_plane_1, vector_plane_2) / np.linalg.norm(np.cross(vector_plane_1, vector_plane_2))
    u = P1 - P0 
    w = P0 - Q0 
    #Проверка на параллельность
    if np.dot(n, u) != 0:
        Si = np.dot(-n, w) / np.dot(n, u)
        intersection = P0 + Si * u
        return intersection
    else:
        return None
def check_intersection(sun, cloude, flower):
    l = LineString([flower.position(),sun.position()]) #Вектор от солнца к цветку
    p = Polygon(cloude.position()[0:3]) #полигон точек, задающих облако
    c= intersection(l, p) #Точка пересечения облака и солнца
    if c is not None:
        if (cloude.get_x()[2] <= c[0] <= cloude.get_x()[0] and cloude.get_y()[1] <= c[1] <= cloude.get_y()[0]):
            return True
    return False

In [ ]:
def draw(sun,cloude_list, flower, t, x_sight, y_sight , z_sight):#Функция рисования
    fig = plt.figure()#создание фигуры - поля для графика
    ax = fig.gca(projection = '3d')#задание 3D варианта осей
    #Параметры оси X
    ax.tick_params(axis='x', which='major', direction='inout',
                bottom=True, top=False, left=True, right=False, 
                color='b', labelcolor='g',
                labelbottom=True, labeltop=False, labelleft=True, labelright=False)
    #Параметры оси Y
    ax.tick_params(axis='y', which='major', direction='inout',
                bottom=True, top=False, left=True, right=False, 
                color='r', labelcolor='red',
                labelbottom=True, labeltop=False, labelleft=True, labelright=False)
    #отрисовка вектора луча солнца
    ax.plot([flower.position()[0],sun.position()[0]],
            [flower.position()[1],sun.position()[1]], 
            [flower.position()[2],sun.position()[2]], 
            color = "y")
    #отрисовка облаков
    for i in range(len(cloude_list)):
        x = cloude_list[i].get_x()
        y = cloude_list[i].get_y()
        z = cloude_list[i].get_z() 
        ax.plot_trisurf(x,y,z, cmap = cm.jet)
    #границы осей
    ax.set_xlim(-x_sight, x_sight)
    ax.set_ylim(-y_sight, y_sight)
    ax.set_zlim(0, z_sight)
    #угол наклона изображения
    ax.view_init(42, -55)
    plt.xlabel('X')
    plt.ylabel('Y')
    fig.savefig(f'Гифка/GIF{t}.png') #Сохранение в папку с кадрами для создания gif-видео
def gif(number_of_iter):
    path = 'C:/Users/Snowfall/Desktop/МАИ/Архитектура информационных систем/Умная лампа/Гифка' #Путь к папке с кадрами
    os.chdir(path) #смена директории
    imgs = [cv2.imread(f"GIF{t}.png") for t in range(number_of_iter)] #чтение всех кадров
    height, width, layers = imgs[0].shape #размеры фото
    video = cv2.VideoWriter(r'gif.mp4',-1,1,(width,height)) #объекта класса для создания видео из библиотеки open cv
    [video.write(i) for i in imgs] #создание видео из кадров
    cv2.destroyAllWindows()
    video.release()

In [ ]:
if __name__ = '__main__':#исполняемый файл
    #параметры mqtt-соединения
    broker_address="farmer.cloudmqtt.com"
    topic = "/test/nfc" 
    login = "njelhmbg"
    pas = "FLLRBnPB4UiD"
    
    number_of_iter = 5
    count_of_cloude = 20
    x_sight = 30
    y_sight = 30
    z_sight = 30
    
    cloude_list = list()
    time_on = list()
    status = list()
    
    date = datetime.datetime.now().timetuple() #установка даты и времени
    #установка параметров, необходимых для расчет угла наклона солнца и азимутального угла
    #в случае неверно заданных параметров устанавливаются параметры город Москва, Волоколамское шоссе 4(МАИ)
    try:
        latitude = float(input("Введите широту(Москва - 55.7522): "))
    except ValueError:
        latitude = 55.55
    try:
        longitude = float(input("Введите долготу(Москва - 37.6156): "))
    except ValueError:
        longitude = 37.37
    try: 
        utc_offset = int(input("Введите временной сдвиг(Москва - 3 часа): "))
    except ValueError:
        utc_offset = 3
    try:
        hour = int(input("Введите время в часах: "))
    except ValueError:
        hour = date[3]
    try:
        minute = int(input("Введите время в минутах: "))
    except ValueError:
        minute = date[4]
    SEA = math.radians(getSEA(latitude, longitude, utc_offset, hour, minute))
    AZ = math.radians(getAZ(latitude, longitude, utc_offset, hour, minute))
    
    flower = Flower()
    sun = Sun(SEA, AZ)
    lamp = Lamp(broker_address, topic, login, pas)
    for _ in range(count_of_cloude): #инициализация списка облаков
        cloude = Cloude()
        cloude_list.append(cloude)
    #основная часть работы программы
    for t in range(number_of_iter):
        time_on.append(t+1)
        if sun.get_z()<0 :
            lamp.on()
        else:
            list_intersection = [check_intersection(sun,cloude_list[i],flower) for i in range(len(cloude_list))] 
            if True in list_intersection:
                lamp.on()
            else:
                lamp.off()
        cloude_list = [cloude for cloude in cloude_list if cloude.check_out_of_sight(x_sight, y_sight)==False]
        status.append(lamp.check())
        draw(sun,cloude_list, flower, t, x_sight, y_sight, z_sight)
        update_field(sun, cloude_list)
        print("Коордитаны солнца: {}".format(sun.position()))  
        time.sleep(1)
    df = pd.DataFrame({'Время с начала запуска': time_on, 'Состояние системы': status})#создание таблица с историей работы лампы
    if(getpass.getuser() == "Snowfall"):#В случае работы программы на моем копьютере
        df.to_excel('Отчет о работе ламы - 3D.xlsx')#сохранение таблицы в Excel-file
    else:
        df.to_excel('Desktop/Отчет о работе ламы - 3D.xlsx')
    gif(number_of_iter)#Создание gif-видео
    df.head(10)#Отображение таблицы